In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations for the training and validation datasets
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the dataset

# Load the dataset
train_dataset = datasets.ImageFolder(root="/content/dataset/InvertDataset/train", transform=transform_train)
val_dataset = datasets.ImageFolder(root="/content/dataset/InvertDataset/valid", transform=transform_val)
test_dataset = datasets.ImageFolder(root="/content/dataset/InvertDataset/test", transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Load the pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

# Freeze the parameters of the pre-trained layers
for param in model.parameters():
    param.requires_grad = False

# Replace the fully connected layer to match the number of classes in your dataset
num_classes = len(train_dataset.classes)  # Update with the actual number of classes in your dataset
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Training and validation loop
epochs = 10
for epoch in range(epochs):
    # Training phase
    model.train()
    train_loss = 0.0
    train_correct = 0
    total_train = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        train_correct += (predicted == labels).sum().item()
        total_train += labels.size(0)

    train_loss /= total_train
    train_accuracy = train_correct / total_train

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    total_val = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            total_val += labels.size(0)

    val_loss /= total_val
    val_accuracy = val_correct / total_val

    print(f"Epoch {epoch + 1}/{epochs}:")
    print(f"  Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
    print(f"  Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

# Save the trained model
torch.save(model.state_dict(), "resnet50_transfer_learning.pth")

# Testing phase
model.eval()
test_loss = 0.0
test_correct = 0
total_test = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        test_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        test_correct += (predicted == labels).sum().item()
        total_test += labels.size(0)

test_loss /= total_test
test_accuracy = test_correct / total_test

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 178MB/s]


Epoch 1/10:
  Train Loss: 1.1849, Train Accuracy: 0.7340
  Val Loss: 0.4581, Val Accuracy: 0.8780
Epoch 2/10:
  Train Loss: 0.4156, Train Accuracy: 0.8866
  Val Loss: 0.3188, Val Accuracy: 0.8900
Epoch 3/10:
  Train Loss: 0.3052, Train Accuracy: 0.9129
  Val Loss: 0.3497, Val Accuracy: 0.8920
Epoch 4/10:
  Train Loss: 0.2316, Train Accuracy: 0.9315
  Val Loss: 0.2970, Val Accuracy: 0.9200
Epoch 5/10:
  Train Loss: 0.2005, Train Accuracy: 0.9407
  Val Loss: 0.3171, Val Accuracy: 0.8920
Epoch 6/10:
  Train Loss: 0.1798, Train Accuracy: 0.9455
  Val Loss: 0.3383, Val Accuracy: 0.8940
Epoch 7/10:
  Train Loss: 0.1569, Train Accuracy: 0.9525
  Val Loss: 0.3094, Val Accuracy: 0.9060
Epoch 8/10:
  Train Loss: 0.1349, Train Accuracy: 0.9577
  Val Loss: 0.4121, Val Accuracy: 0.8840
Epoch 9/10:
  Train Loss: 0.1274, Train Accuracy: 0.9599
  Val Loss: 0.3942, Val Accuracy: 0.9040
Epoch 10/10:
  Train Loss: 0.1115, Train Accuracy: 0.9646
  Val Loss: 0.3227, Val Accuracy: 0.8960
Test Loss: 0.3271, 